### Introduction

This notebook is created and shared to help people learn and understand the process of solving a problem which involves text variables. Apart from creating new variables, you'll learn to extract ~650 text (count) features and use them in training a xgboost model. This script scores ~0.70 on public leaderboard.
For any questions, feel free to raise issues.

In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
# load libraries

import pandas as pd
import numpy as np
import re
import datetime
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb

pd.set_option('display.max_colwidth',100)

In [3]:
#load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
# convert unix time format
unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: datetime.datetime.fromtimestamp(int(k)).strftime('%Y-%m-%d %H:%M:%S'))
    test[x] = test[x].apply(lambda k: datetime.datetime.fromtimestamp(int(k)).strftime('%Y-%m-%d %H:%M:%S'))


### Some features

In [5]:
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    train[len_feats[i]] = train[cols_to_use[i]].apply(str).apply(len)
    test[len_feats[i]] = test[cols_to_use[i]].apply(str).apply(len)

In [6]:
train['name_count'] = train['name'].str.split().str.len()
train['desc_count'] = train['desc'].str.split().str.len()

test['name_count'] = test['name'].str.split().str.len()
test['desc_count'] = test['desc'].str.split().str.len()

In [7]:
train['keywords_len'] = train['keywords'].str.len()
train['keywords_count'] = train['keywords'].str.split('-').str.len()

test['keywords_len'] = test['keywords'].str.len()
test['keywords_count'] = test['keywords'].str.split('-').str.len()

### Some more features

In [8]:
# converting string variables to datetime
unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))
    test[x] = test[x].apply(lambda k: datetime.datetime.strptime(k, '%Y-%m-%d %H:%M:%S'))

In [9]:
# there should be simpler way - might take longer
# creating list with time difference between 1) launched_at and created_at 2) deadline and launched_at

time1 = []
time3 = []
time0 = []
for i in np.arange(train.shape[0]):
    time1.append(np.round((train.loc[i, 'launched_at'] - train.loc[i, 'created_at']).total_seconds()).astype(int))
    time3.append(np.round((train.loc[i, 'deadline'] - train.loc[i, 'launched_at']).total_seconds()).astype(int))
    time0.append(np.round((train.loc[i, 'state_changed_at'] - train.loc[i, 'deadline']).total_seconds()).astype(int))

In [10]:
train['time1'] = np.log(time1)
train['time3'] = np.log(time3)
train['time0'] = np.log(time0)

In [11]:
# for test data
time5 = []
time6 = []
time7 = []
for i in np.arange(test.shape[0]):
    time5.append(np.round((test.loc[i, 'launched_at'] - test.loc[i, 'created_at']).total_seconds()).astype(int))
    time6.append(np.round((test.loc[i, 'deadline'] - test.loc[i, 'launched_at']).total_seconds()).astype(int))
    time7.append(np.round((test.loc[i, 'state_changed_at'] - test.loc[i, 'deadline']).total_seconds()).astype(int))

In [12]:
test['time1'] = np.log(time5)
test['time3'] = np.log(time6)
test['time0'] = np.log(time7)

In [13]:
feat = ['disable_communication','country']

for x in feat:
    le = LabelEncoder()
    le.fit(list(train[x].values) + list(test[x].values))
    train[x] = le.transform(list(train[x]))
    test[x] = le.transform(list(test[x]))

In [14]:
train['goal'] = np.log1p(train['goal'])
test['goal'] = np.log1p(test['goal'])

### Text Cleaning

In [15]:
# creating a full list of descriptions from train and etst
kickdesc = pd.Series(train['desc'].tolist() + test['desc'].tolist()).astype(str)

In [16]:
# this function cleans punctuations, digits and irregular tabs. Then converts the sentences to lower
def desc_clean(word):
    p1 = re.sub(pattern='(\W+)|(\d+)|(\s+)',repl=' ',string=word)
    p1 = p1.lower()
    return p1

kickdesc = kickdesc.map(desc_clean)

In [17]:
stop = set(stopwords.words('english'))
kickdesc = [[x for x in x.split() if x not in stop] for x in kickdesc]

stemmer = SnowballStemmer(language='english')
kickdesc = [[stemmer.stem(x) for x in x] for x in kickdesc]

kickdesc = [[x for x in x if len(x) > 2] for x in kickdesc]

kickdesc = [' '.join(x) for x in kickdesc]

### Creating Count Features

In [18]:
# Due to memory error, limited the number of features to 650
cv = CountVectorizer(max_features=650)

In [19]:
alldesc = cv.fit_transform(kickdesc).todense()

In [20]:
#create a data frame
combine = pd.DataFrame(alldesc)
combine.rename(columns= lambda x: 'variable_'+ str(x), inplace=True)

In [21]:
#split the text features

train_text = combine[:train.shape[0]]
test_text = combine[train.shape[0]:]

test_text.reset_index(drop=True,inplace=True)

### Finalizing train and test data before merging

In [22]:
cols_to_use = ['name_len','desc_len','keywords_len','name_count','desc_count','keywords_count','time1','time3','time0','goal']

In [23]:
target = train['final_status']

In [24]:
train = train.loc[:,cols_to_use]
test = test.loc[:,cols_to_use]

In [25]:
X_train = pd.concat([train, train_text],axis=1)
X_test = pd.concat([test, test_text],axis=1)

In [26]:
print(X_train.shape)
print(X_test.shape)

(108129, 660)
(63465, 660)


In [27]:
test.columns

Index(['name_len', 'desc_len', 'keywords_len', 'name_count', 'desc_count',
       'keywords_count', 'time1', 'time3', 'time0', 'goal'],
      dtype='object')

In [28]:
test_id = pd.read_csv('test.csv')
test_id = test_id['project_id']

### Model Training

In [29]:
from xgboost.sklearn import XGBClassifier

In [30]:
xgb = XGBClassifier(learning_rate =0.025, n_estimators=1000, max_depth=6,
                    min_child_weight=5, gamma=0.01, subsample=0.7, colsample_bytree=0.7,
                    objective= 'binary:logistic', nthread=4, seed=2017)

In [31]:
xgb.fit(X_train,target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0.01, learning_rate=0.025,
       max_delta_step=0, max_depth=6, min_child_weight=5, missing=None,
       n_estimators=1000, nthread=4, objective='binary:logistic',
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2017,
       silent=True, subsample=0.7)

In [32]:
pred = xgb.predict(X_test)

In [33]:
submission = pd.DataFrame({'project_id':test_id,'final_status':pred})
submission = submission[['project_id','final_status']]
submission.to_csv("xgb.csv",index = False)

In [34]:
submission.head()

,project_id,final_status
0,kkst917493670,0
1,kkst1664901914,0
2,kkst925125077,0
3,kkst1427645275,0
4,kkst1714249266,0
